# Demonstration of `df_subgroups_states2summary_df.py`

Demonstrating use of `df_subgroups_states2summary_df.py`, see [here](https://github.com/fomightez/text_mining) for more information.

This script comverts a dataframe or data table of text into a summary.

-----

The two main ways of using the script are covered, featuring several of the options available.

## Preparation and displaying USAGE block

Let's get the script and run 'Help' on it to see the basic USAGE block.

(If you are running this notebook in the session launched from the repo that includes the script, this step is not necessary. However, it is included because there is no harm in running it here and you may be wanting to run this elsewhere or see how to easily acquire the script. If you are on the actual command line, you'd leave off the exclamation point.)

In [1]:
import os
file_needed = "df_subgroups_states2summary_df.py"
if not os.path.isfile(file_needed):
    !curl -OL https://raw.githubusercontent.com/fomightez/text_mining/master/df_subgroups_states2summary_df.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22686  100 22686    0     0   119k      0 --:--:-- --:--:-- --:--:--  119k


In [2]:
%run df_subgroups_states2summary_df.py -h

usage: df_subgroups_states2summary_df.py [-h] [-ord ORDER] [-olsp] [-bc]
                                         DF_FILE GROUPS SUBGROUPS

df_subgroups_states2summary_df.py takes a dataframe, and some information
about columns in the dataframe and makes a summary data table with the
percents for each subgrouping / state per total and each group. **** Script by
Wayne Decatur (fomightez @ github) ***

positional arguments:
  DF_FILE               Name of file containing the dataframe. Whether it is
                        in the form of a pickled dataframe, tab-separated
                        text, or comma-separated text needs to be indicated by
                        the file extension. So `.pkl`, `.tsv`, or `.csv` for
                        the file extension.
  GROUPS                Text indicating column in dataframe to use as main
                        grouping categories.
  SUBGROUPS             Text indicating column in dataframe to use as
                        subgroupi

## Use the script by calling it from the command line

A dataframe  or text data table will be used for input data. To fully demonstrate the options for the script we'll use a toy dataframe and also convert it to a text table.

In [3]:
import pandas as pd
sales = [('Jones LLC', 177887, 'yes'),
         ('Jones LLC', 12387, 'yes'),
         ('Jones LLC', 1772287, 'yes'),
         ('Jones LLC', 1187, 'maybe'),
         ('Jones LLC', 1773297, 'maybe'),
         ('Jones LLC', 19897, 'no'),
         ('Alpha Co', 157987, 'yes'),
         ('Alpha Co', 158981, 'yes'),
         ('Alpha Co', 159983, 'yes'),
         ('Alpha Co', 167987, 'yes'),
         ('Alpha Co', 158117, 'yes'),
         ('Alpha Co', 1999917, 'maybe'),
         ('Alpha Co', 193917, 'maybe'),
         ('Alpha Co', 1933917, 'maybe'),
         ('Alpha Co', 159333, 'no'),
         ('Alpha Co', 256521, 'no'),
         ('Blue Inc', 111947, 'no')]
labels = ['Manufacturer', 'Item', 'In_Stock']
df = pd.DataFrame.from_records(sales, columns=labels)
df.head()

,Manufacturer,Item,In_Stock
0,Jones LLC,177887,yes
1,Jones LLC,12387,yes
2,Jones LLC,1772287,yes
3,Jones LLC,1187,maybe
4,Jones LLC,1773297,maybe


Let's save that dataframe as tabular text and also as a Pickled pickled dataframe. The former being human readable and the latter not. The latter is more efficient at storeage though if that is an issue.

First to save as tabular text in tab-separated form. You could change it to be comma-separated, CSV, if you choose.

In [4]:
df.to_pickle("data.pkl")
df.to_csv('data.tsv', sep='\t',index = False)

Now that we have files with input data, we have something we can point the script at for running it.

In addition to providing the data input file name, the text corresponding to the column heading of the groupings and the text corresponding to the column containing the subgroups have to be provided when calling the script.

In [5]:
%run df_subgroups_states2summary_df.py data.pkl Manufacturer In_Stock

Summary dataframe saved as a text table easily opened in
different software; file named: `summary_data.tsv`

Summary dataframe saved in pickled form for ease of use within
Python; file named: `summary_data.pkl`. This will retain the column headers/names formatting best.

In [6]:
t = pd.read_pickle("summary_data.pkl")

In [7]:
t

yes           maybe              no          
          [n] count         % count         % count         %
ALL        17     8  0.470588     5  0.294118     4  0.235294
Alpha Co   10     5  0.500000     3  0.300000     2  0.200000
Blue Inc    1     0  0.000000     0  0.000000     1  1.000000
Jones LLC   6     3  0.500000     2  0.333333     1  0.166667

In [8]:
# This would change the view to be nicer; note the underlying dataframe remains unotuched
t_styl = t.style.format("{:.2%}",subset=[('yes','%'),('maybe','%'),('no','%')]) # based on https://stackoverflow.com/a/56411982/8508004
# and https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.format.html
# Trick to add a title to the dataframe
from IPython.display import display, HTML
# trick from https://stackoverflow.com/a/29665452/8508004
display(HTML('<b>Items in stock by Manufacturer:</b>'))
display(t_styl)

In [9]:
t.columns

MultiIndex(levels=[['', 'maybe', 'no', 'yes'], ['%', '[n]', 'count']],
           codes=[[0, 3, 3, 1, 1, 2, 2], [1, 2, 0, 2, 0, 2, 0]])

In [10]:
%run df_subgroups_states2summary_df.py data.pkl Manufacturer In_Stock --order no,yes,maybe

Summary dataframe saved as a text table easily opened in
different software; file named: `summary_data.tsv`

Summary dataframe saved in pickled form for ease of use within
Python; file named: `summary_data.pkl`. This will retain the column headers/names formatting best.

In [11]:
o = pd.read_pickle("summary_data.pkl")
o

no             yes           maybe          
          [n] count         % count         % count         %
ALL        17     4  0.235294     8  0.470588     5  0.294118
Alpha Co   10     2  0.200000     5  0.500000     3  0.300000
Blue Inc    1     1  1.000000     0  0.000000     0  0.000000
Jones LLC   6     1  0.166667     3  0.500000     2  0.333333

In [12]:
# This would change the view to be nicer; note the underlying dataframe remains unotuched
o_styl = o.style.format("{:.2%}",subset=[('yes','%'),('maybe','%'),('no','%')]) # based on https://stackoverflow.com/a/56411982/8508004
# and https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.format.html
o_styl

In [13]:
%run df_subgroups_states2summary_df.py data.pkl Manufacturer In_Stock --only_subgrp_perc

Summary dataframe saved as a text table easily opened in
different software; file named: `summary_data.tsv`

Summary dataframe saved in pickled form for ease of use within
Python; file named: `summary_data.pkl`. This will retain the column headers/names formatting best.

In [14]:
m = pd.read_pickle("summary_data.pkl")
m

,,yes,maybe,no
,[n],%,%,%
ALL,17,0.470588,0.294118,0.235294
Alpha Co,10,0.500000,0.300000,0.200000
Blue Inc,1,0.000000,0.000000,1.000000
Jones LLC,6,0.500000,0.333333,0.166667


In [15]:
# This would change the view to be nicer; note the underlying dataframe remains unotuched
m_styl = m.style.format("{:.2%}",subset=[('yes','%'),('maybe','%'),('no','%')]) # based on https://stackoverflow.com/a/56411982/8508004
# and https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.formats.style.Styler.format.html
m_styl

,,yes,maybe,no
,[n],%,%,%
ALL,17,47.06%,29.41%,23.53%
Alpha Co,10,50.00%,30.00%,20.00%
Blue Inc,1,0.00%,0.00%,100.00%
Jones LLC,6,50.00%,33.33%,16.67%


In [21]:
def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)
ms = m.style.apply(highlight_max, subset=['yes', 'maybe', 'no'])
ms

,,yes,maybe,no
,[n],%,%,%
ALL,17,0.470588,0.294118,0.235294
Alpha Co,10,0.5,0.3,0.2
Blue Inc,1,0,0,1
Jones LLC,6,0.5,0.333333,0.166667


In [16]:
%run df_subgroups_states2summary_df.py data.pkl Manufacturer In_Stock  --bracket_counts

Summary dataframe saved as a text table easily opened in
different software; file named: `summary_data.tsv`. This version meant for presenation only.

Summary dataframe saved in pickled form for ease of use within
Python; file named: `summary_data.pkl`. This version meant for
presentation only.


**Also saving data table as forms easier to handle for subsequent steps:**
Summary dataframe saved as a text table easily opened in
different software; file named: `summary_basic_data.tsv`

Summary dataframe saved in pickled form for ease of use within
Python; file named: `summary_basic_data.pkl`. This will retain the column headers/names formatting best.

In [17]:
bc = pd.read_pickle("summary_data.pkl")
bc

,[n],yes,maybe,no
ALL,17.0,47.06% [8],29.41% [5],23.53% [4]
Alpha Co,10.0,50.00% [5],30.00% [3],20.00% [2]
Blue Inc,1.0,0.00% [0],0.00% [0],100.00% [1]
Jones LLC,6.0,50.00% [3],33.33% [2],16.67% [1]


In [18]:
import time

def executeSomething():
    #code here
    print ('.')
    time.sleep(480) #60 seconds times 8 minutes

while True:
    executeSomething()

.


KeyboardInterrupt: 